In [66]:
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch

In [67]:
torchvision.datasets.MNIST('data', train=True, transform=ToTensor(), download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [68]:
train_dataset = torchvision.datasets.MNIST('data', train=True, transform=ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST('data', train=False, transform=ToTensor(),download=True)

In [69]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [70]:
imgs, label = next(iter(train_dataloader))

In [71]:
imgs.shape

torch.Size([64, 1, 28, 28])

# 创建模型

In [72]:
model = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))

In [73]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Model().to(device)
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=0.001)

# 训练函数

In [74]:
def train(dataloader, Model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    train_loss, train_correct = 0, 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            train_correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()
            train_loss += loss.item()
    train_correct /= size
    train_loss /= num_batch
    return train_correct, train_loss

# 测试函数

In [75]:
def test(dataloader, Model, loss_fn):
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    test_loss, test_correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            test_correct += (y_pred.argmax(1) == y).type(torch.float).sum().item()
            test_loss += loss.item()
    test_correct /= size
    test_loss /= num_batch
    return test_correct, test_loss

# 训练与验证

In [76]:
epoches = 100
train_loss = []
train_correct = []
test_loss = []
test_correct = []
for epoch in range(epoches):
    epoch_train_correct, epoch_train_loss = train(train_dataloader, model, loss_fn, opt)
    epoch_test_correct, epoch_test_loss = test(test_dataloader, model, loss_fn)
    train_loss.append(epoch_train_loss)
    train_correct.append(epoch_train_correct)
    test_loss.append(epoch_test_loss)
    test_correct.append(epoch_test_correct)
    template = ("epoch:{:2d}, train_loss:{:.5f}, train_correct:{:.2f}%, test_loss:{:.5f}, test_correct:{:.2f}%")
    print(template.format(epoch, epoch_train_loss, epoch_train_correct*100, epoch_test_loss, epoch_test_correct*100))

epoch: 0, train_loss:2.30084, train_correct:9.75%, test_loss:2.29824, test_correct:9.74%
epoch: 1, train_loss:2.29612, train_correct:9.75%, test_loss:2.29346, test_correct:9.74%
epoch: 2, train_loss:2.29093, train_correct:9.80%, test_loss:2.28748, test_correct:10.03%
epoch: 3, train_loss:2.28368, train_correct:12.77%, test_loss:2.27824, test_correct:17.00%
epoch: 4, train_loss:2.27161, train_correct:25.64%, test_loss:2.26206, test_correct:38.09%
epoch: 5, train_loss:2.24872, train_correct:46.58%, test_loss:2.22875, test_correct:52.01%
epoch: 6, train_loss:2.19483, train_correct:51.65%, test_loss:2.13975, test_correct:54.85%
epoch: 7, train_loss:2.01333, train_correct:56.95%, test_loss:1.78465, test_correct:63.56%
epoch: 8, train_loss:1.36089, train_correct:70.34%, test_loss:0.92251, test_correct:78.16%
epoch: 9, train_loss:0.75384, train_correct:80.09%, test_loss:0.59116, test_correct:83.67%
epoch:10, train_loss:0.55065, train_correct:84.31%, test_loss:0.47373, test_correct:86.60%
epoc

epoch:91, train_loss:0.06310, train_correct:98.11%, test_loss:0.06499, test_correct:98.03%
epoch:92, train_loss:0.06242, train_correct:98.06%, test_loss:0.06637, test_correct:97.92%
epoch:93, train_loss:0.06185, train_correct:98.17%, test_loss:0.06226, test_correct:98.09%
epoch:94, train_loss:0.06171, train_correct:98.12%, test_loss:0.06144, test_correct:98.11%
epoch:95, train_loss:0.06085, train_correct:98.17%, test_loss:0.06287, test_correct:98.10%
epoch:96, train_loss:0.06048, train_correct:98.17%, test_loss:0.06103, test_correct:98.11%
epoch:97, train_loss:0.05997, train_correct:98.22%, test_loss:0.06118, test_correct:98.09%
epoch:98, train_loss:0.05936, train_correct:98.18%, test_loss:0.05994, test_correct:98.20%
epoch:99, train_loss:0.05893, train_correct:98.25%, test_loss:0.05899, test_correct:98.23%


# 封装fit函数

In [77]:
def fit(epoches, train_dataloader, test_dataloader, model, loss_fn, opt):
    train_loss = []
    train_correct = []
    test_loss = []
    test_correct = []
    for epoch in range(epoches):
        epoch_train_correct, epoch_train_loss = train(train_dataloader, model, loss_fn, opt)
        epoch_test_correct, epoch_test_loss = test(test_dataloader, model, loss_fn)
        train_loss.append(epoch_train_loss)
        train_correct.append(epoch_train_correct)
        test_loss.append(epoch_test_loss)
        test_correct.append(epoch_test_correct)
        template = ("epoch:{:2d}, train_loss:{:.5f}, train_correct:{:.2f}%, test_loss:{:.5f}, test_correct:{:.2f}%")
        print(template.format(epoch, epoch_train_loss, epoch_train_correct*100, epoch_test_loss, epoch_test_correct*100))
        return train_loss, train_correct, test_loss, test_correct